# Convolutional Neural Network

Problem: Given a list of handwritten digit images and their labels, train a model to recognize digits 
    with a convolutional neural network

In [1]:
# tensorflow for R library
library(tensorflow)

In [2]:
# fetch data
datasets <- tf$contrib$learn$datasets
mnist <- datasets$mnist$read_data_sets("MNIST-data", one_hot = TRUE)

In [3]:
# input place holder
x <- tf$placeholder(tf$float32, shape(NULL, 28L * 28L))
x_image <- tf$reshape(x, shape(-1L, 28L, 28L, 1L))
# output place holder
y_ <- tf$placeholder(tf$float32, shape(NULL, 10L))

In [4]:
# utility function for weight variable
weight_variable <- function(shape) {
    initial <- tf$truncated_normal(shape, stddev=0.1)
    tf$Variable(initial)
}

# utility function for bias variable
bias_variable <- function(shape) {
    initial <- tf$constant(0.1, shape=shape)
    tf$Variable(initial)
}

# utility function for convolution 2D
conv2d <- function(x, W) {
    tf$nn$conv2d(x, W, strides=c(1L, 1L, 1L, 1L), padding='SAME')
}

# utility function for max pooling
max_pool_2x2 <- function(x) {
    tf$nn$max_pool(x, ksize=c(1L, 2L, 2L, 1L), strides=c(1L, 2L, 2L, 1L), padding='SAME')
}

In [5]:
# convolution model

# first layer 5x5 with 32 filters
W_conv1 <- weight_variable(shape(5L, 5L, 1L, 32L))
b_conv1 <- bias_variable(shape(32L))
h_conv1 <- tf$nn$relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 <- max_pool_2x2(h_conv1)

# second layer 5x5 with 64 features
W_conv2 <- weight_variable(shape = shape(5L, 5L, 32L, 64L))
b_conv2 <- bias_variable(shape = shape(64L))
h_conv2 <- tf$nn$relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 <- max_pool_2x2(h_conv2)

# image size reduced to 7x7, add a fully connected layer with 1024 neurons
W_fc1 <- weight_variable(shape(7L * 7L * 64L, 1024L))
b_fc1 <- bias_variable(shape(1024L))
h_pool2_flat <- tf$reshape(h_pool2, shape(-1L, 7L * 7L * 64L))
h_fc1 <- tf$nn$relu(tf$matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout layer
keep_prob <- tf$placeholder(tf$float32)
h_fc1_drop <- tf$nn$dropout(h_fc1, keep_prob)

# softmax layer
W_fc2 <- weight_variable(shape(1024L, 10L))
b_fc2 <- bias_variable(shape(10L))
y_conv <- tf$nn$softmax(tf$matmul(h_fc1_drop, W_fc2) + b_fc2)

In [6]:
# entropy and optimizer
cross_entropy <- tf$reduce_mean(-tf$reduce_sum(y_ * tf$log(y_conv), reduction_indices=1L))
train_step <- tf$train$AdamOptimizer(1e-4)$minimize(cross_entropy)

# accuracy
correct_prediction <- tf$equal(tf$argmax(y_conv, 1L), tf$argmax(y_, 1L))
accuracy <- tf$reduce_mean(tf$cast(correct_prediction, tf$float32))

# session
sess = tf$Session()
sess$run(tf$global_variables_initializer())

# training loop
for (i in 1:2000) {
    # mini batch
    batch <- mnist$train$next_batch(50L)
    if (i %% 20 == 0) {
        train_accuracy <- sess$run(accuracy, feed_dict = dict(x = batch[[1]], y_ = batch[[2]], keep_prob = 1.0))
        cat(sprintf("step %d, training accuracy %g\n", i, train_accuracy))
    }    
    sess$run(train_step, feed_dict = dict(x = batch[[1]], y_ = batch[[2]], keep_prob = 0.5))
}


step 20, training accuracy 0.5
step 40, training accuracy 0.78
step 60, training accuracy 0.8
step 80, training accuracy 0.76
step 100, training accuracy 0.92
step 120, training accuracy 0.92
step 140, training accuracy 0.92
step 160, training accuracy 0.84
step 180, training accuracy 0.88
step 200, training accuracy 0.92
step 220, training accuracy 0.86
step 240, training accuracy 0.88
step 260, training accuracy 0.94
step 280, training accuracy 0.9
step 300, training accuracy 0.96
step 320, training accuracy 0.88
step 340, training accuracy 0.94
step 360, training accuracy 0.98
step 380, training accuracy 0.96
step 400, training accuracy 0.92
step 420, training accuracy 1
step 440, training accuracy 1
step 460, training accuracy 0.92
step 480, training accuracy 0.94
step 500, training accuracy 0.88
step 520, training accuracy 0.94
step 540, training accuracy 0.92
step 560, training accuracy 0.98
step 580, training accuracy 0.96
step 600, training accuracy 0.98
step 620, training accu

In [7]:
# test accuracy
test_accuracy <- sess$run(accuracy, feed_dict = dict(x = mnist$test$images, y_ = mnist$test$labels, keep_prob = 1.0))
cat(sprintf("test accuracy %g", test_accuracy))

test accuracy 0.9748